# Data generation

# Source data:

Eurostat purchasing power parity data - https://ec.europa.eu/eurostat/web/purchasing-power-parities/data/database

Wise API - Currency conversion rates

# Categories
The country-country comparisons will be calculated for the following expense categories:
- Alcoholic beverages & Tobacco
- Clothing and footwear
- Education
- Food and non-alcoholic beverages
- Health
- Recreation, culture, restaurants & hotels
- Software, communication & IT
- Transport
- Water, electricity, gas and other fuels

In [116]:
import pandas as pd
import numpy as np

import calc
import itertools

import json

In [110]:
data_filelocation = "prc_ppp_ind_1_Data.csv"
currency_rate_filelocation = "rates.json"
supported_countries = {
    "Austria": "at",
    "Denmark": "dk",
    "France": "fr",
    "Germany (until 1990 former territory of the FRG)": "de",
    "Hungary": "hu",
    "Spain": "es",
    "Sweden": "se",
}
currencies = {
    "at" : "EUR",
    "dk" : "DKK",
    "fr" : "EUR",
    "de" : "EUR",
    "hu" : "HUF",
    "es" : "EUR",
    "se" : "SEK",
}
category_names = {
    "alc-tobacco": ["Alcoholic beverages, tobacco and narcotics"],
    "clothing": ["Clothing and footwear"],
    "education": ["Education"],
    "food-non-alc": ["Food and non-alcoholic beverages"],
    "health": ["Health"],
    "it-tech": ["Communication", "Software", "Audio-visual, photographic and information processing equipment"],
    "leisure": ["Restaurants and hotels", "Recreation and culture"],
    "transport": ["Transport"],
}

In [111]:
df = pd.read_csv(data_filelocation, encoding="cp1252")

with open(currency_rate_filelocation, "r") as file:
    currency_conversion_data = json.load(file)

In [97]:
# Remove countries not on supported list
df = df[df["GEO"].isin(supported_countries.keys())]

# Remap country name to country_id
df['country_id'] = df['GEO'].map(supported_countries)

# Keep only columns I need
df.drop(columns=["TIME", "GEO", "NA_ITEM"], inplace=True)

# Convert Value to float
df["Value"] = df["Value"].astype(float)

df.head(10)

,PPP_CAT,Value,country_id
48,Food and non-alcoholic beverages,8.92214,dk
49,"Alcoholic beverages, tobacco and narcotics",9.95311,dk
50,Clothing and footwear,9.92872,dk
51,"Electricity, gas and other fuels",9.80323,dk
52,Health,11.09850,dk
53,Transport,9.50216,dk
54,Communication,7.72018,dk
55,Recreation and culture,10.37910,dk
56,"Audio-visual, photographic and information pro...",7.36770,dk
57,Education,11.80400,dk


In [98]:
df2 = pd.DataFrame(columns=["country_id", "category", "value"])

for _, country_id in supported_countries.items():
    for category, subcategory_list in category_names.items():
        temp_df = df[(df["country_id"] == country_id) & (df["PPP_CAT"].isin(subcategory_list))]

        # If result has just 1 row, just get top value
        if len(subcategory_list) == 1:
            value = temp_df["Value"].iloc[0]
        
        # If result contains mutiple things, call calculation
        else:
            value = calc.calc_category_index(temp_df["Value"].iloc[:])
        
        df2.loc[len(df2)] = [country_id, category, value]

In [101]:
df2.head(10)

,country_id,category,value
0,at,alc-tobacco,0.876951
1,at,clothing,1.031500
2,at,education,1.751530
3,at,food-non-alc,1.070830
4,at,health,1.449080
5,at,it-tech,0.991853
6,at,leisure,1.150035
7,at,transport,1.090980
8,dk,alc-tobacco,9.953110
9,dk,clothing,9.928720


In [108]:
out_df = pd.DataFrame(columns=["From", "To"])

# Create all country to-from combination for outdf
countries = currencies.keys()

for combination in itertools.permutations(countries, 2):
    out_df.loc[len(out_df)] = [combination[0], combination[1]]

out_df

,From,To
0,at,dk
1,at,fr
2,at,de
3,at,hu
4,at,es
5,at,se
6,dk,at
7,dk,fr
8,dk,de
9,dk,hu


In [118]:
# Get currency data for countries
x = calc.exchange_rate_finder(currency_conversion_data, "EUR", "HUF")

x

AttributeError: module 'calc' has no attribute 'exchange_rate_finder'